In [ ]:
import cv2
import math
import time
import argparse

In [2]:
def getFaceBox(net,frame,conf_threshold=0.7):
    frameopencvdnn=frame.copy()
    frameheight=frameopencvdnn.shape[0]
    framewidth=frameopencvdnn.shape[1]
    blob=cv2.dnn.blobFromImage(frameopencvdnn,1.0,(300,300),[104,117,123],True,False)
    net.setInput(blob)
    detection=net.forward()
    bboxes=[]
    for i in range(detection.shape[2]):
        confidence=detection[0,0,i,2]
        if confidence > conf_threshold:
            x1=int(detection[0,0,i,3] * framewidth)
            y1=int(detection[0,0,i,4]* frameheight)
            x2=int(detection[0,0,i,5]* framewidth)
            y2=int(detection[0,0,i,6]* frameheight)
            bboxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameopencvdnn,(x1,y1),(x2,y2),(0,255,0),int(round(frameheight/150)),8)
    return frameopencvdnn,bboxes
    
    

In [3]:
faceproto="opencv_face_detector.pbtxt.txt"
facemodel="opencv_face_detector_uint8.pb"
ageproto="age_deploy.prototxt.txt"
agemodel="age_net.caffemodel"
genderproto="gender_deploy.prototxt.txt"
gendermodel="gender_net.caffemodel"

MODEL_MEAN_VALUES=(78.4263377603,87.7689143744,114.895847746)
ageList=['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']
genderlist=['Male','Female']

In [4]:
agenet=cv2.dnn.readNet(agemodel,ageproto)
gendernet=cv2.dnn.readNet(gendermodel,genderproto)
facenet=cv2.dnn.readNet(facemodel,faceproto)

In [5]:
cap=cv2.VideoCapture(0)
padding=20
while True:
    hasframe,frame=cap.read()
    
    if not hasframe:
        cv2.waitKey()
        break
    
    small_frame=cv2.resize(frame,(500,500))
    frameface,bboxes= getFaceBox(facenet,small_frame)
    if not bboxes:
        print("no face detected,checking next frame")
        continue
    for bbox in bboxes:
        face=small_frame[max(0,bbox[1]-padding):min(bbox[3]+padding,frame.shape[0]-1),max(0,bbox[0]-padding):min(bbox[2]+padding,frame.shape[1]-1)]
        blob=cv2.dnn.blobFromImage(face,1.0,(227,227),MODEL_MEAN_VALUES,swapRB=False)
        gendernet.setInput(blob)
        genderpreds=gendernet.forward()
        gender=genderlist[genderpreds[0].argmax()]
        print("gender :{},conf={:.3f}".format(gender,genderpreds[0].max()) )
        
        agenet.setInput(blob)
        agepreds=agenet.forward()
        age=ageList[agepreds[0].argmax()]
        print("age :{},conf={:.3f}".format(age,agepreds.max()))
        
              
        label="{},{}".format(age,gender)
        cv2.putText(frameface,label,(bbox[0],bbox[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2,cv2.LINE_AA)
        cv2.imshow('Age gender Demo',frameface)
    if cv2.waitKey(1) & 0xff==ord("q"):
        cap.realse()
        cv2.destroyAllWindows()
        break

gender :Male,conf=0.985
age :(8-12),conf=0.423
gender :Male,conf=1.000
age :(25-32),conf=0.492
gender :Male,conf=1.000
age :(25-32),conf=0.350
gender :Male,conf=1.000
age :(25-32),conf=0.796
gender :Male,conf=1.000
age :(8-12),conf=0.578
gender :Male,conf=1.000
age :(25-32),conf=0.692
gender :Male,conf=1.000
age :(25-32),conf=0.397
gender :Male,conf=1.000
age :(25-32),conf=0.809
gender :Male,conf=1.000
age :(25-32),conf=0.586
gender :Male,conf=1.000
age :(8-12),conf=0.571
gender :Male,conf=1.000
age :(15-20),conf=0.531
gender :Male,conf=1.000
age :(25-32),conf=0.553
gender :Male,conf=1.000
age :(25-32),conf=0.602
gender :Male,conf=1.000
age :(25-32),conf=0.599
gender :Male,conf=1.000
age :(25-32),conf=0.783
gender :Male,conf=1.000
age :(25-32),conf=0.543
gender :Male,conf=1.000
age :(25-32),conf=0.809
gender :Male,conf=1.000
age :(25-32),conf=0.585
gender :Male,conf=1.000
age :(25-32),conf=0.725
gender :Male,conf=1.000
age :(25-32),conf=0.560
gender :Male,conf=1.000
age :(25-32),conf=0

no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
no face detected,checking next frame
gender :Male,conf=0.916
age :(8-12),conf=0.676
gender :Male,conf=0.955
age :(38-43),conf=0.559
gender :Male,conf=0.985
age :(8-12),conf=0.983
gender :Male,conf=0.965
age :(8-12),conf=0.988
gender :Male,conf=1.000
age :(8-12),conf=0.886
gender :Male,conf=1.000
age :(8-12),conf=0.990
gender :Male,conf=1.000
age :(8-12),conf=0.926
gender :Male,conf=1.000
age :(8-12),conf=0.926
gender :Male,conf=0.999
age :(8-12),conf=0.989
gender :Male,conf=0.9

gender :Male,conf=1.000
age :(25-32),conf=0.787
gender :Male,conf=1.000
age :(25-32),conf=0.694
gender :Male,conf=1.000
age :(25-32),conf=0.725
gender :Male,conf=1.000
age :(25-32),conf=0.737
gender :Male,conf=1.000
age :(25-32),conf=0.562
gender :Male,conf=1.000
age :(25-32),conf=0.540
gender :Male,conf=1.000
age :(25-32),conf=0.502
gender :Male,conf=1.000
age :(25-32),conf=0.813
gender :Male,conf=1.000
age :(8-12),conf=0.993
gender :Male,conf=1.000
age :(8-12),conf=0.984
gender :Male,conf=1.000
age :(8-12),conf=0.985
gender :Male,conf=1.000
age :(8-12),conf=0.998
gender :Male,conf=1.000
age :(8-12),conf=0.968
gender :Male,conf=1.000
age :(8-12),conf=0.997
gender :Male,conf=1.000
age :(8-12),conf=0.996
gender :Male,conf=1.000
age :(8-12),conf=0.947
gender :Male,conf=0.999
age :(8-12),conf=0.818
gender :Male,conf=0.999
age :(8-12),conf=0.881
gender :Male,conf=1.000
age :(8-12),conf=0.918
gender :Male,conf=1.000
age :(8-12),conf=0.962
gender :Male,conf=1.000
age :(8-12),conf=0.649
gende

age :(8-12),conf=0.486
gender :Male,conf=1.000
age :(25-32),conf=0.529
gender :Male,conf=1.000
age :(25-32),conf=0.498
gender :Male,conf=1.000
age :(25-32),conf=0.631
gender :Male,conf=1.000
age :(25-32),conf=0.558
gender :Male,conf=1.000
age :(8-12),conf=0.533
gender :Male,conf=1.000
age :(25-32),conf=0.595
gender :Male,conf=1.000
age :(25-32),conf=0.483
gender :Male,conf=1.000
age :(25-32),conf=0.484
gender :Male,conf=1.000
age :(25-32),conf=0.570
gender :Male,conf=1.000
age :(25-32),conf=0.548
gender :Male,conf=1.000
age :(8-12),conf=0.847
gender :Male,conf=1.000
age :(8-12),conf=0.880
gender :Male,conf=1.000
age :(8-12),conf=0.730
gender :Male,conf=1.000
age :(8-12),conf=0.803
gender :Male,conf=1.000
age :(8-12),conf=0.536
gender :Male,conf=1.000
age :(8-12),conf=0.647
gender :Male,conf=1.000
age :(8-12),conf=0.624
gender :Male,conf=1.000
age :(8-12),conf=0.785
gender :Male,conf=1.000
age :(8-12),conf=0.813
gender :Male,conf=1.000
age :(8-12),conf=0.510
gender :Male,conf=1.000
age 

age :(8-12),conf=0.970
gender :Male,conf=1.000
age :(8-12),conf=0.883
gender :Male,conf=1.000
age :(8-12),conf=0.981
gender :Male,conf=1.000
age :(8-12),conf=0.979
gender :Male,conf=1.000
age :(8-12),conf=0.973
gender :Male,conf=1.000
age :(8-12),conf=0.971
gender :Male,conf=1.000
age :(8-12),conf=0.954
gender :Male,conf=1.000
age :(8-12),conf=0.915
gender :Male,conf=1.000
age :(15-20),conf=0.421
gender :Male,conf=1.000
age :(25-32),conf=0.741
gender :Male,conf=1.000
age :(8-12),conf=0.608
gender :Male,conf=1.000
age :(38-43),conf=0.633
gender :Male,conf=1.000
age :(25-32),conf=0.821
gender :Male,conf=1.000
age :(25-32),conf=0.926
gender :Male,conf=1.000
age :(8-12),conf=0.710
gender :Male,conf=1.000
age :(8-12),conf=0.831
gender :Male,conf=1.000
age :(38-43),conf=0.510
gender :Male,conf=1.000
age :(38-43),conf=0.519
gender :Male,conf=0.999
age :(8-12),conf=0.882
gender :Male,conf=1.000
age :(25-32),conf=0.441
gender :Male,conf=1.000
age :(8-12),conf=0.680
gender :Male,conf=1.000
age :

gender :Male,conf=1.000
age :(8-12),conf=0.819
gender :Male,conf=1.000
age :(15-20),conf=0.542
gender :Male,conf=1.000
age :(8-12),conf=0.799
gender :Male,conf=1.000
age :(8-12),conf=0.799
gender :Male,conf=1.000
age :(8-12),conf=0.695
gender :Male,conf=1.000
age :(8-12),conf=0.887
gender :Male,conf=1.000
age :(8-12),conf=0.923
gender :Male,conf=1.000
age :(8-12),conf=0.898
gender :Male,conf=1.000
age :(8-12),conf=0.955
gender :Male,conf=1.000
age :(8-12),conf=0.953
gender :Male,conf=1.000
age :(8-12),conf=0.853
gender :Male,conf=1.000
age :(8-12),conf=0.987
gender :Male,conf=1.000
age :(8-12),conf=0.998
gender :Male,conf=1.000
age :(8-12),conf=0.999
gender :Male,conf=1.000
age :(8-12),conf=0.996
gender :Male,conf=0.998
age :(8-12),conf=0.983
gender :Male,conf=1.000
age :(8-12),conf=0.928
gender :Male,conf=1.000
age :(8-12),conf=0.809
gender :Male,conf=1.000
age :(8-12),conf=0.815
gender :Male,conf=1.000
age :(8-12),conf=0.963
gender :Male,conf=1.000
age :(8-12),conf=0.803
gender :Male

AttributeError: 'cv2.VideoCapture' object has no attribute 'realse'

In [1]:
print(0xff)

255


In [2]:
ord('q')

113